<a href="https://colab.research.google.com/github/abdulsamadkhan/Reasoning/blob/main/GRPO_with_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRPO with Unsloth

## installing libraries   

In [1]:
!pip install unsloth vllm
!pip install --upgrade pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16

##Setting up Unsloth

In [2]:
from unsloth import FastLanguageModel

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Now, let’s load the Llama 3.1 8B Instruct model and configure it for fine-tuning:

---

This code loads the model in 4-bit quantization to save memory and applies LoRA (Low-Rank Adaptation) for efficient fine-tuning. The target_modules parameter specifies which layers of the model to fine-tune, and use_gradient_checkpointing enables training with longer contexts.

---

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024  # Can increase for longer reasoning traces
lora_rank = 32  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=True,  # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6,  # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Remove QKVO if out of memory
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",  # Enable long context finetuning
    random_state=3407,
)

INFO 03-15 05:30:37 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3. vLLM: 0.7.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.31%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 17.29 GB. Also swap space = 6 GB.
INFO 03-15 05:30:56 config.py:549] This model supports multiple tasks: {'embed', 'generate', 're

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 03-15 05:32:32 model_runner.py:1562] Graph capturing finished in 52 secs, took 0.87 GiB
INFO 03-15 05:32:32 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 65.90 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Preparation
First, we will define the format of the prompts and answers:



In [4]:
# Define the system prompt that instructs the model to use a specific format
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

Now, let’s prepare the dataset:

In [5]:
import re
from datasets import load_dataset, Dataset


# Helper functions to extract answers from different formats
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()


def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()


# Function to prepare the GSM8K dataset
def get_gsm8k_questions(split="train") -> Dataset:
    data = load_dataset("openai/gsm8k", "main")[split]
    data = data.map(
        lambda x: {
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": x["question"]},
            ],
            "answer": extract_hash_answer(x["answer"]),
        }
    )
    return data


dataset = get_gsm8k_questions()

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [6]:
dataset['prompt'][2]

[{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
  'role': 'system'},
 {'content': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?',
  'role': 'user'}]

In [7]:
from transformers import AutoTokenizer

# Define the prompt
prompt = dataset["prompt"][2]


# Extract the content from the prompt dictionaries
text = "".join([d["content"] for d in prompt])

# Tokenize the input using the extracted text
inputs = tokenizer(text, return_tensors="pt").to("cuda")  # Move to GPU

# Generate a response
with torch.no_grad():  # No gradients needed for inference
    output_ids = model.generate(**inputs, max_length=256, temperature=0.7, top_p=0.9)

# Decode the output
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)


Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet? 

<reasoning>
Betty has half of the money she needs, which is $50. Her parents give her $15, and her grandparents give her twice as much as her parents, which is $30. So, Betty has $50 + $15 + $30 = $95. Betty needs $100 - $95 = $5 more.
</reasonition>
<answer>
$5
</answer> 

This problem is a good example of a real-world application of basic arithmetic operations. It requires the student to apply the concepts of addition and subtraction to solve a practical problem. The problem also requires the student to understand the concept of half and twice as much, which is an important aspect of fractions and ratios. 

This problem is appropriate for 4t

## Defining Reward Functions

Different reward functions
*  **correctness_reward_func**	Rewards the model when its
answer matches the correct answer
* **int_reward_func	Rewards** the model for providing a numeric answer
* **strict_format_reward_func and soft_format_reward_func**	Reward the model for following the specified format
*  **xmlcount_reward_func**	Rewards proper XML tag usage and penalizes extra content after the closing tags

In [8]:

# Reward function that checks if the answer is correct
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]["content"] for completion in completions]
    q = prompts[0][-1]["content"]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print(
        "-" * 20,
        f"Question:\n{q}",
        f"\nAnswer:\n{answer[0]}",
        f"\nResponse:\n{responses[0]}",
        f"\nExtracted:\n{extracted_responses[0]}",
    )
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


# Reward function that checks if the answer is an integer
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]["content"] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]


# Reward function that checks if the completion follows the strict format
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


# Reward function that checks if the completion follows a more relaxed format
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


# Reward function that counts XML tags and penalizes extra content
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count


def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

## Training with GRPO


The GRPOConfig sets various hyperparameters for training:

* `use_vllm:` Enables fast inference with vLLM
* `learning_rate:` Controls how quickly the model learns
* `num_generations:` Number of completions to generate for each prompt
* `max_steps:` Total number of training steps to perform

In [9]:
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256
training_args = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=6,  # Decrease if out of memory
    max_prompt_length=max_prompt_length,
    max_completion_length=max_seq_length - max_prompt_length,
    # num_train_epochs = 1,  # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


In [10]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
Let's break it down step by step:

1. The first 10 tickets cost full price, which is 10 * $40 = $400.
2. The remaining 2 tickets exceed the 10 tickets and qualify for a 5% discount. 
    The discount for each ticket is 5% of $40, which is 0.05 * $40 = $2.
    So the price for each exceeding ticket is $40 - $2 = $38.
3. The total cost of the 2 tickets with the 5% discount is 2 * $38 = $76.
4. The total amount Mr. Benson paid is the cost of the 10 full-price tickets plus the cost of the 2 discounted tickets.
    Total = $400 + $76 = $476.

<answer> $476 </answer> 
Extracted:
$476


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,-0.089667,0.219638,177.333344,0.000000,-0.089667,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,353.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.130167,0.202155,210.333344,0.000324,-0.130167,0.000000,0.000000,0.000000,0.000000
4,0.000000,1.161833,1.300059,190.166672,0.000214,-0.088167,0.000000,0.000000,0.250000,1.000000
5,0.000000,0.317667,0.938759,135.500000,0.000661,-0.099000,0.000000,0.000000,0.083333,0.333333
6,0.000000,1.026167,1.431642,235.666672,0.000234,-0.223833,0.000000,0.000000,0.250000,1.000000
7,0.000000,0.000000,0.000000,284.333344,0.000343,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.150000,0.327552,162.500000,0.000619,-0.150000,0.000000,0.000000,0.000000,0.000000
9,0.000000,1.569500,1.254238,164.166672,0.000394,-0.180500,0.000000,0.000000,0.416667,1.333333
10,0.000000,0.000000,0.000000,250.833344,0.000320,0.000000,0.000000,0.000000,0.000000,0.000000


Streaming output truncated to the last 5000 lines.
Lee had $10 and his friend had $8. So they had a total of $10 + $8 = $18.

To find the change, subtract the total amount spent from the amount they had. 

Change = $18 - $15 = $3

<answer> $3 </answer> 
Extracted:
$3
-------------------- Question:
Andy walks 50 meters from his house to school. After school, he comes back to the house and goes to the market. If he walks 140 meters in total, how many meters is the distance between the house and the market? 
Answer:
40 
Response:
Let's break it down:

Andy walks 50 meters from his house to school.
To get back home, he walks 50 meters, so the distance from the school to his house is also 50 meters.
The total distance walked from house to school and back is 50 + 50 = 100 meters.

Now, we know that the total distance walked in a day is 140 meters, and we already know that 100 meters is the distance from home to school and back.
To find the distance between the house and the market, we subtra

TrainOutput(global_step=250, training_loss=0.0010853310808832842, metrics={'train_runtime': 2837.328, 'train_samples_per_second': 0.529, 'train_steps_per_second': 0.088, 'total_flos': 0.0, 'train_loss': 0.0010853310808832842})

##Testing the Model
After training, let’s test our model to see how it performs. First, we’ll save the LoRA weights

In [ ]:
model.save_lora("grpo_saved_lora")

test the model on a new question  

In [ ]:
from vllm import SamplingParams

text = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": "Calculate pi."},
    ],
    tokenize=False,
    add_generation_prompt=True,
)

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)
output = (
    model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_saved_lora"),
    )[0]
    .outputs[0]
    .text
)

print(output)

# Saving the Model
Unsloth provides several options for saving your fine-tuned model, but we’ll focus on the most common.

In [11]:
# Save to 16-bit precision
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 44.67 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 16/32 [00:00<00:00, 53.01it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:19<00:00,  1.60it/s]


Unsloth: Saving tokenizer... Done.
Done.


## Pushing to Hugging Face Hub
We’ll push the model to the Hugging Face Hub using the push_to_hub_merged method. This method allows us to push the model in multiple quantization formats.

In [13]:

from huggingface_hub import login

# Log in to Hugging Face Hub
login()

In [14]:
# Push to Hugging Face Hub (requires a token)
model.push_to_hub_merged(
    "abdulsamad/laamaInstruct_tuned", tokenizer, save_method="merged_16bit"
)

Unsloth: You are pushing to hub, but you passed your HF username = abdulsamad.
We shall truncate abdulsamad/laamaInstruct_tuned to laamaInstruct_tuned
100%|██████████| 32/32 [00:20<00:00,  1.55it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/625 [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Letus test the model on the training instances

In [16]:

# Define the prompt
prompt = dataset["prompt"][2]


# Extract the content from the prompt dictionaries
text = "".join([d["content"] for d in prompt])

# Tokenize the input using the extracted text
inputs = tokenizer(text, return_tensors="pt").to("cuda")  # Move to GPU

# Generate a response
with torch.no_grad():  # No gradients needed for inference
    output_ids = model.generate(**inputs, max_length=256, temperature=0.7, top_p=0.9)

# Decode the output
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)

##Conclusion
In this exercise, you’ve learned how to:

1. Set up Unsloth for accelerated fine-tuning
2. Prepare data for GRPO training
3. Define custom reward functions to guide the model’s learning
4. Train a model using GRPO
5. Test the fine-tuned model
6. Save the model in various formats